# �� Classification Iris avec Triton Inference Server

Ce notebook démontre l'utilisation de NVIDIA Triton Inference Server pour déployer et servir un modèle de classification Iris.

## Configuration de l'environnement
- **Image**: `s2i-generic-data-science-notebook:2025.1`
- **Namespace**: `triton-demo`
- **User**: `mouachan`
- **Base URL**: `/notebook/triton-demo/test-triton`

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

print("🔧 Configuration de l'environnement...")
print(f"📁 Répertoire de travail: {os.getcwd()}")
print(f"👤 Utilisateur: {os.getenv('USER', 'unknown')}")
print(f"🏷️ Namespace: {os.getenv('NAMESPACE', 'unknown')}")

# Variables d'environnement du workbench
print(f"\n🔗 Configuration du workbench:")
print(f"   JUPYTER_IMAGE: {os.getenv('JUPYTER_IMAGE', 'Non configuré')}")

# Variables Model Registry et S3
print(f"\n📊 Configuration Model Registry:")
print(f"   MODEL_REGISTRY_URL: {os.getenv('MODEL_REGISTRY_URL', 'Non configuré')}")

print(f"\n☁️ Configuration S3/MinIO:")
print(f"   AWS_ACCESS_KEY_ID: {os.getenv('AWS_ACCESS_KEY_ID', 'Non configuré')}")
print(f"   AWS_S3_ENDPOINT: {os.getenv('AWS_S3_ENDPOINT', 'Non configuré')}")
print(f"   AWS_S3_BUCKET: {os.getenv('AWS_S3_BUCKET', 'Non configuré')}")

# Créer les dossiers nécessaires
os.makedirs('models', exist_ok=True)
os.makedirs('data', exist_ok=True)
print("\n✅ Dossiers créés")

In [ ]:
# Charger le dataset Iris
print("📊 Chargement du dataset Iris...")
iris = load_iris()
X = iris.data
y = iris.target

print(f"📈 Forme des données: {X.shape}")
print(f"🎯 Nombre de classes: {len(np.unique(y))}")
print(f"🏷️ Classes: {iris.target_names}")
print(f"📋 Features: {iris.feature_names}")

# Diviser en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\n📊 Division train/test:")
print(f"   Train: {X_train.shape[0]} échantillons")
print(f"   Test: {X_test.shape[0]} échantillons")

In [ ]:
# Entraîner le modèle
print("🤖 Entraînement du modèle Random Forest...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prédictions et évaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\n📊 Performance du modèle:")
print(f"   Accuracy: {accuracy:.4f}")
print(f"   Classes: {iris.target_names}")

# Rapport de classification
print("\n📋 Rapport de classification:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

In [ ]:
# Visualisation des données
plt.figure(figsize=(15, 5))

# Distribution des classes
plt.subplot(1, 3, 1)
sns.countplot(y=y_train)
plt.title('Distribution des classes (Train)')
plt.xlabel('Classe')
plt.ylabel('Nombre d\'échantillons')

# Corrélation entre features
plt.subplot(1, 3, 2)
df_train = pd.DataFrame(X_train, columns=iris.feature_names)
df_train['target'] = y_train
sns.heatmap(df_train.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Matrice de corrélation')

# Importance des features
plt.subplot(1, 3, 3)
feature_importance = pd.DataFrame({
    'feature': iris.feature_names,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=True)
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.title('Importance des features')
plt.xlabel('Importance')

plt.tight_layout()
plt.show()

In [ ]:
# Sauvegarder le modèle
import pickle
from datetime import datetime

# Sauvegarder le modèle scikit-learn
model_path = 'models/iris_classifier.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

# Sauvegarder les métadonnées
model_metadata = {
    'accuracy': float(accuracy),
    'model_type': 'RandomForestClassifier',
    'features': iris.feature_names,
    'classes': iris.target_names.tolist(),
    'n_estimators': model.n_estimators,
    'training_date': datetime.now().isoformat(),
    'dataset_size': {
        'train': len(X_train),
        'test': len(X_test),
        'total': len(X)
    }
}

import json
metadata_path = 'models/model_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(model_metadata, f, indent=2)

print(f"✅ Modèle sauvegardé: {model_path}")
print(f"✅ Métadonnées sauvegardées: {metadata_path}")
print("\n📋 Métadonnées du modèle:")
for key, value in model_metadata.items():
    print(f"   {key}: {value}")

In [ ]:
print("🎉 Résumé de l'exécution:")
print(f"   📊 Dataset: {X.shape[0]} échantillons, {X.shape[1]} features")
print(f"   🤖 Modèle: Random Forest ({model.n_estimators} arbres)")
print(f"   📈 Accuracy: {accuracy:.4f}")
print(f"   💾 Modèle: {model_path}")
print(f"   📋 Métadonnées: {metadata_path}")

print("\n�� Prochaines étapes:")
print("   1. Convertir le modèle au format ONNX")
print("   2. Déployer le modèle sur Triton")
print("   3. Tester l'inférence en temps réel")
print("   4. Enregistrer dans le Model Registry")

print("\n🔧 Configuration du workbench:")
print(f"   📁 Working Directory: {os.getcwd()}")
print(f"   🏷️ Namespace: {os.getenv('NAMESPACE', 'unknown')}")
print(f"   👤 User: {os.getenv('USER', 'unknown')}")
print(f"   🔗 Base URL: /notebook/triton-demo/test-triton")